In [83]:
# !pip install pyodbc#<-- iNSTALAR PYODBC

import pandas as pd
import pyodbc # <-- BOM PARA LEMBRAR PARA COLOCAR NO PORTFOLIO 

In [84]:
server = 'DESKTOP-KLPI1BO\\SQLEXPRESS' # Substitua pelo nome do servidor SQL Server ***ATENÇÃO NA BARRA DUPLA
database = 'car_shop'  # Substitua pelo nome do banco de dados
conexaoDB = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};' # PARA ACHAR ESSA LINHA PROCURAR EM FONTES DE DADOS ODBC E ABA DRIVERS
                      f'SERVER={server};'
                      f'DATABASE={database};'
                      'Trusted_Connection=yes;')

cursor = conexaoDB.cursor()   # criando cursor de comando 

In [85]:
dados = pd.read_excel(r"C:\Users\Luiz Krawiec\OneDrive\Documentos\portfolio\automação, implantação e EDA\excel\tbFunnel.xlsx")
#display(dados)
dados.head(5)
#display(dados)

,visit_id,customer_id,store_id,product_id,visit_page_date,add_to_cart_date,start_checkout_date,finish_checkout_date,paid_date,discount
0,38836BE8B968,73650706FFAE,BF580E604866,produto001,2021-02-19,2021-02-19,NaT,NaT,NaT,-0.34
1,7FB675C2F431,236A739A6063,F79B06D22BC3,produto002,2021-04-20,NaT,NaT,NaT,NaT,-0.38
2,B07A4A5759AC,CF2DFCBE74E3,B06DF477008E,produto003,2021-06-25,NaT,NaT,NaT,NaT,-0.14
3,713A295F9E4C,25504A496AEB,97D85A358174,produto004,2021-06-04,2021-06-04,NaT,NaT,NaT,-0.08
4,D6145FA8A291,05DE6BCDFEC7,BF580E604866,produto005,2021-08-26,NaT,NaT,NaT,NaT,-0.30


In [86]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30580 entries, 0 to 30579
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   visit_id              30580 non-null  object        
 1   customer_id           30580 non-null  object        
 2   store_id              30580 non-null  object        
 3   product_id            30580 non-null  object        
 4   visit_page_date       30580 non-null  datetime64[ns]
 5   add_to_cart_date      20020 non-null  datetime64[ns]
 6   start_checkout_date   4198 non-null   datetime64[ns]
 7   finish_checkout_date  3807 non-null   datetime64[ns]
 8   paid_date             3788 non-null   datetime64[ns]
 9   discount              30580 non-null  float64       
dtypes: datetime64[ns](5), float64(1), object(4)
memory usage: 2.3+ MB


In [87]:
str(dados.columns).replace("'","") 

'Index([visit_id, customer_id, store_id, product_id, visit_page_date,\n       add_to_cart_date, start_checkout_date, finish_checkout_date,\n       paid_date, discount],\n      dtype=object)'

In [88]:
cursor.execute('truncate table[tbFunnel6]')   #executa tarefa de  apagar dados
cursor.commit()

In [89]:
for index, linha in dados.iterrows():
    # Converter as datas para o formato de data do SQL Server
    visit_page_date = linha['visit_page_date'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(linha['visit_page_date']) else None
    add_to_cart_date = linha['add_to_cart_date'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(linha['add_to_cart_date']) else None
    start_checkout_date = linha['start_checkout_date'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(linha['start_checkout_date']) else None
    finish_checkout_date = linha['finish_checkout_date'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(linha['finish_checkout_date']) else None
    paid_date = linha['paid_date'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(linha['paid_date']) else None
    
    # Inserir os valores na tabela
    cursor.execute("INSERT INTO [tbFunnel6] (visit_id, customer_id, store_id, product_id, visit_page_date, add_to_cart_date, start_checkout_date, finish_checkout_date, paid_date, discount) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   (str(linha['visit_id']), str(linha['customer_id']), str(linha['store_id']), str(linha['product_id']), visit_page_date, add_to_cart_date, start_checkout_date, finish_checkout_date, paid_date, linha['discount']))

# Comitar as mudanças
conexaoDB.commit()

# Fechar cursor e conexão
cursor.close()
conexaoDB.close()


In [90]:
cursor.execute('delete  from [tbFunnel6]')   #executa tarefa de  apagar dados
cursor.commit()